In [2]:
#Now we will look at working with one of the mostl populate NoSQL database i.e. MongoDB.
# Python uses pymongo library that has the official mongo db APIs. Mongita is basically a subset of the pymongo implementation.

#Migrating from mongita to mongoDB

#install pymongo

%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# We will reuse the mongita implementation of the crypto currency manager to work with MongoDB.
#We just need to switch the client to use MongoDB and that's it.



import requests
import click
from pymongo import MongoClient #create client

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}


# Establish database connection
client = MongoClient() #default connects to localhost:27017
database = client.new_portfolio
investments = database.investments


#CLick is the pythonic way to create a command line interface for your scripts or application. It's smart and easy to use and creates arguments automatically for the decorators defined.
# Under the hood it has a CLI API that has a commandline parser that talks with the interpreter than works completely different than optparse or argparse built in libraries. It gives you easy ways to create highly nested and complex commands with options.
# CLick also provides out of the box support for reading env variables form values and lazy loading and file handling.
# You transform methods into click commands using decorators.

#Create a command group, a group can have multiple commands assocated with it. The group name will the function name
@click.group()
def cli():
    pass

def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]

@cli.command() #Command decorator that aslo assigns this command to the group.
@click.option("--coin_id", default="bitcoin") #Adding an option
@click.option("--currency", default="usd")
def get_investment_value(coin_id, currency): #the option name gets passed onto the function parameter name. Then the function uses iut wthin the code.
    coin_price = get_coin_price(coin_id, currency)
    filter_doc_buy = {
        "coin_id": coin_id,
        "currency": currency,
        "sell": "False"
    }
    filter_doc_sell = {
        "coin_id": coin_id,
        "currency": currency,
        "sell": "True"
    }
    buy_result = investments.find(filter_doc_buy)
    sell_result = investments.find(filter_doc_sell)
    
    buy_amount = sum(row["amount"] for row in buy_result) 
    sell_amount = sum(row["amount"] for row in sell_result)
    
    total = buy_amount - sell_amount
    
    print(f"You own a total of {total} {coin_id} worth {total * coin_price} {currency.upper()}")

@cli.command()
@click.option("--coin_id", default="bitcoin")
@click.option("--currency", default="usd")
def show_coin_price(coin_id, currency):
    coin_price = get_coin_price(coin_id, currency)
    print(f"The price of {coin_id} is {coin_price:.2f} {currency.upper()}")
    
@cli.command()
@click.option("--coin_id")
@click.option("--currency")
@click.option("--amount", type=float)
@click.option("--sell", is_flag=True)
def add_investments(coin_id, currency, amount, sell):
    try:
        # sql = "INSERT INTO investments (coin_id, currency, amount, sell, date) VALUES (?, ?, ?, ?, ?);"
        investments.insert_one({
            "coin_id": coin_id,
            "currency": currency,
            "amount": amount,
            "sell": f"{True if sell else False}"
        })
        print(f"Added {'sell' if sell else 'buy'} of {amount} {coin_id}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
@cli.command()
def show_db():
    results = investments.find({})
    [print(result) for result  in results]

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass

# Example usage in Jupyter
run_command('show-coin-price', '--coin_id', 'ethereum', '--currency', 'usd')
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '1.0')
#selling some btc
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '0.5', '--sell')

#Getting investment value
run_command('get-investment-value', '--coin_id', 'bitcoin', '--currency', 'inr')

run_command('show-db')

The price of ethereum is 2521.68 USD
Added buy of 1.0 bitcoin
Added sell of 0.5 bitcoin
You own a total of 1.0 bitcoin worth 9081083.0 INR
{'_id': ObjectId('6844af2044970eccd3b07961'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 1.0, 'sell': 'False'}
{'_id': ObjectId('6844af2044970eccd3b07962'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 0.5, 'sell': 'True'}
{'_id': ObjectId('6844af4544970eccd3b07964'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 1.0, 'sell': 'False'}
{'_id': ObjectId('6844af4544970eccd3b07965'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 0.5, 'sell': 'True'}


In [8]:
#Mongo DB enhanced features.

#Till now you've seen mongita implmentation which is a file based embedded implementation of Mongo DB. 
# The full Mongo DB database offfers a lot of enahanced capabilities.

# Let's see Filtering multiple conditions. Something that mongita find() method couild not do. 
# 
# Let's see below Mongita query call

# investments.update_one(
#     {"coin_id": "bitcoin"}, #this was the filter docuemnt. In Mongita, it cannot take mutiple conditions like, coin_id=bitcoin and amount>2
#     {"$inc", {"amount": 1}} # This was the update document that takes in $inc operator and {"amount": 1} is the update document to be applied.
    
# )

#Mongo DB supports $and operator in the filter document. That means it will look to satfisty multiple conditions.

#the $and operator accepts a list of filter doucuments and apply them all.


results = investments.find(
    {
        "$and": # $and operator accepts a list of documents to process
        [ 
            {"coin_id": "bitcoin"},
            {"amount": {"$gt": 0}} # $gt operator means greater than. Operators in pymongo are special key's of dictionary.
        ]
    }
)

[print(result["amount"]) for result in results]

1.0
0.5
1.0
0.5


[None, None, None, None]

In [ ]:
#Relationships in MongoDB, using embedded documents.

# Just like in JSON, a value of akey can be another object.
# {
#     "key_1": "value_1",
#     "key_2": {
#         "embedded_key1": "embedded_value1",
#         "embedded_key2": "embedded_value2",
#         "embedded_key3": "embedded_value3",
#     },
#     "key_3": "value_3"
# }

#Similarly we can embed documents within documents to create relationships.

{'key_1': 'value_1',
 'key_2': {'embedded_key1': 'embedded_value1',
  'embedded_key2': 'embedded_value2',
  'embedded_key3': 'embedded_value3'},
 'key_3': 'value_3'}

In [ ]:
# Final implementation including embedded documents and relationships.
# We now have a watchlist document in our database that has a list of coins as investments, and some metadata.

import click, datetime
from pymongo import MongoClient #create client
from bson import ObjectId
from MongoDemoUtils import get_coin_price, seed_data as utils_seed_data


# Establish database connection
client = MongoClient() #default connects to localhost:27017
database = client.new_portfolio
watchlists = database.watchlists #Create a watchlist collection

#Helper methods

def _get_all_watchlist_names():
    return list(watchlists.find({},{"name": 1})) #Even though you set a projection document specefying you just want a name attribute, it will still have object Id of the docuemnt in the Cursor object

def _select_watchlist(watchlist_names): #Takes in a cursor object with just names of the watchlist
    for index, watchlist in enumerate(watchlist_names):
        print(f"{index + 1}: {watchlist['name']} ")
    selected_watchlist_index = int(input("Select a watchlist: ")) - 1
    selected_watchlist_id = watchlist_names[selected_watchlist_index]["_id"] #_id attribute stores the document ID
    return watchlists.find_one({"_id": ObjectId(selected_watchlist_id)}) #find_one returns a dictionary directly rather than a Cursor object.

def _select_coin_from_watchlist(watchlist):
    for index, coin in enumerate(watchlist['coins']):
        print(f"{index + 1}: {coin['coin']} ")
    selected_coin_index = int(input("Select a coin: ")) - 1
    return watchlist['coins'][selected_coin_index]

def _add_coin_to_watchlist(watchlist_oid, coin):
    filter = {"_id": ObjectId(watchlist_oid)}
    update = {"$push": {"coins": coin}} # $push operator actually adds the document passed to a list within the doucment
    watchlists.update_one(filter,update)    
    
def _remove_a_coin_from_watchlist(watchlist_oid, coin):
    filter = {"_id": ObjectId(watchlist_oid)}
    update = {"$pull": {"coins": coin}}
    watchlists.update_one(filter,update)

@click.group()
def cli():
    pass

@cli.command(help="Seed the database.")
@click.option("--force", is_flag=True, help="Seed even if database is not empty.")
def seed_data(force):
    if force:
        utils_seed_data(watchlists)
    elif watchlists.count_documents({}) > 0:
        print("The database is not empty, use the --force option or the clear-data command")
    else:
        utils_seed_data(watchlists)
    
@cli.command(help = "View the coins and current prices of the data")
def view_watchlists():
    selected_watchlist = _select_watchlist(_get_all_watchlist_names())
    
    print(f"Watchlist: {selected_watchlist['name']} in {selected_watchlist['metadata']['currency']}")
    print(f"Description: {selected_watchlist['metadata']['description']}")
    print(f"{"-" * 25}")
    print("Coins:")
    coin_prices = get_coin_price([coin['coin'] for coin in selected_watchlist['coins']], selected_watchlist['metadata']['currency'])
    for index, coin in enumerate(selected_watchlist['coins']):
        print(f"{str(index + 1).rjust(3, " ")}: {coin['coin']} = {coin['note']}")
        print(f"       Current price: {coin_prices[coin['coin']]} ")
    print("Prices provided by CoinGecko")
        

@cli.command(help="Add a new watchlist to the portfolio")
@click.option("--name", help="Name of the watchlist")
@click.option("--description", help="Description of the watchlist")
@click.option("--currency", help="Currency to display prices in", default="usd")
def add_watchlist(name, description, currency):
    metadata = {
        "description": description,
        "currency": currency,
        "date_added": datetime.datetime.now()
    }
    watchlist = {
        "name": name,
        "metadata": metadata,
        "coins": []
    }
    watchlists.insert_one(watchlist)
    
    print(f"Added {name} watchlist")
    
@cli.command(help="Add a coin to a watchlist")
@click.option("--coin", help="The coin id to add")
@click.option("--note", help="A description about the coin")
def add_coin(coin, note):
    selected_watchlist = _select_watchlist(_get_all_watchlist_names())
    coin_to_add = {
        "coin": coin,
        "note": note,
        "date_added": datetime.datetime.now()
    }
    _add_coin_to_watchlist(selected_watchlist['_id'], coin_to_add)
    print(f"Added {coin} to the {selected_watchlist['name']} watchlist")
    
@cli.command(help="Remove a coin from the watchlist")
def remove_coin():
    selected_watchlist = _select_watchlist(_get_all_watchlist_names())
    selected_coin = _select_coin_from_watchlist(selected_watchlist)
    
    _remove_a_coin_from_watchlist(selected_watchlist["_id"], selected_coin)
    print(f"Removed {selected_coin['coin']} from {selected_watchlist['name']} watchlist")
    
@cli.command()
def show_db():
    results = investments.find({})
    [print(result) for result  in results]

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass
    
@cli.command()
def clear_data():
    database.drop_collection(watchlists)
    print("All data cleared from the watchlists collection")

# # Example usage in Jupyter
run_command('clear-data')
run_command('seed-data')
run_command('view-watchlists')
run_command('add-watchlist', '--name', 'My Watchlist', '--currency', 'usd', '--description', 'My Watchlist')
run_command('add-coin', '--coin', 'dogecoin', '--note', "It's a joke")
run_command('remove-coin')

1: Bulls 
2: Bears 
3: My Watchlist 
Watchlist: Bulls in usd
Description: Coins to buy
-------------------------
Coins:
Constructed crypto_ids: bitcoin
Constructed URL: https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd
  1: bitcoin = The most popular coin
       Current price: 106321 
Prices provided by CoinGecko
